This program logs into Amazon, Pizzahobby and Agrieuro collects the latest sells from past day
and reports them via email.

In [2]:
# import for webscraping
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support.select import Select

import pandas as pd
from datetime import datetime, timedelta
import time


# Imports for sending mail
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


import random
from fake_useragent import UserAgent

# Pizza Hobby Commissions

In [8]:
# Gets commissions from pizza hobby
def pizza_hobby(date_yesterday):

    # login credentials fro pizzahobby.de
    username = "##########" 
    password = "**********"

    # adds options for Chrome WebDriver
    chrome_options = webdriver.ChromeOptions()
    #chrome_options.add_argument('--headless')                   #<-- prevents to open test window
    #chrome_options.add_argument('--no-sandbox')                #<--- don't know what this does
    #chrome_options.add_argument('--disable-dev-shm-usage')      #<--- don't know what this does

    # adds an random useragent to prevent recaptcha detection
    ua = UserAgent()
    userAgent = ua.random
    chrome_options.add_argument(f'user-agent={userAgent}')

    # Opens Chrome Browser
    driver = webdriver.Chrome(chrome_options)
    
    
    time.sleep(5)

    # Minimizes Browser Windows
    # driver.maximize_window()

    # Navigates to URL
    driver.get("https://pizzahobby.de/anmeldung?back=my-account")

    time.sleep(random.uniform(1,10))


    # Selects username field on login page
    username_field_pizza_hobby = driver.find_element("name", "email") 
    # Enters username 
    username_field_pizza_hobby.send_keys(username)

    # Waits for 5 seconds to prevent suspicous behavior
    WebDriverWait(driver=driver, timeout=5)

    # Selects password field on login page
    password_field_pizza_hobby = driver.find_element("name", "password") 
    # Enters password 
    password_field_pizza_hobby.send_keys(password)


    time.sleep(random.uniform(1,10))

    # Click sign in button to login the website.
    driver.find_element(By.XPATH, '//*[@id="submit-login"]').click()

    # Wait for login process to complete and field "Zugehörigkeit" is loaded 
    time.sleep(random.uniform(1,10))

    # Click on field "Zugehörigkeit"
    driver.find_element(By.XPATH, '//*[@id="content"]/div/div/a[8]').click()

    # Wait until page loaded and field "My Statistics" is loaded 
    time.sleep(random.uniform(1,10))

    # Click on field "My Statistics"
    driver.find_element(By.XPATH, '//*[@id="main"]/ul/li[7]/a').click()

    # Waits for 5 seconds until page is loaded 
    time.sleep(random.uniform(1,10))

    # Selects affiliate commission table
    driver.find_element(By.XPATH, '//*[@id="stats"]/div/table')
    
    # Extracts commission table from website
    pizza_hobby_affiliate_commissions_df = pd.read_html(driver.find_element(By.XPATH, '//*[@id="stats"]/div/table').get_attribute('outerHTML'))[0]

    # Closes Webdriver
    driver.close()

    # Changes type of date column to datetime 
    pizza_hobby_affiliate_commissions_df['Date'] = pd.to_datetime(pizza_hobby_affiliate_commissions_df['Date'])

    # Changes timeformat such that only date is displayed
    pizza_hobby_affiliate_commissions_df['Date'] = pizza_hobby_affiliate_commissions_df['Date'].dt.date
    
    # Gets the commissions from yesterday 
    commissions = pizza_hobby_affiliate_commissions_df['Reward Value'].loc[pizza_hobby_affiliate_commissions_df.Date == pd.Timestamp(date_yesterday)]

    # Converts commissions to float
    commissions = (commissions.replace('[\€,]', '', regex=True).astype(float))/100

    # Sums up commissions
    total_commissions_pizza_hobby = commissions.sum()


    return total_commissions_pizza_hobby

# Amazon Commissions


In [22]:
# Gets commissions from pizza hobby
def amazon(date_yesterday):

    # login credentials fro pizzahobby.de
    username = "##########" 
    password = "**********"

    # adds options for Chrome WebDriver
    chrome_options = webdriver.ChromeOptions()
    #chrome_options.add_argument('--headless')                   #<-- prevents to open test window
    #chrome_options.add_argument('--no-sandbox')                #<--- don't know what this does
    #chrome_options.add_argument('--disable-dev-shm-usage')      #<--- don't know what this does

    # adds an random useragent to prevent recaptcha detection
    ua = UserAgent()
    userAgent = ua.random
    chrome_options.add_argument(f'user-agent={userAgent}')

    # Opens Chrome Browser
    driver = webdriver.Chrome(chrome_options)


    time.sleep(5)

    # Minimizes Browser Windows
    # driver.maximize_window()

    # Navigates to URL
    driver.get('https://www.amazon.de/ap/signin?openid.pape.max_auth_age=900&openid.return_to=https%3A%2F%2Fwww.amazon.de%2Fgp%2Fyourstore%2Fhome%3Fpath%3D%252Fgp%252Fyourstore%252Fhome%26signIn%3D1%26useRedirectOnSuccess%3D1%26action%3Dsign-out%26ref_%3Dnav_AccountFlyout_signout&openid.assoc_handle=deflex&openid.mode=checkid_setup&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0')

    time.sleep(random.uniform(3,10))


    # Selects username field on login page
    username_field_pizza_hobby = driver.find_element("name", "email") 
    # Enters username 
    username_field_pizza_hobby.send_keys(username)

    # Waits for 5 seconds to prevent suspicous behavior
    time.sleep(random.uniform(3,10))

    # Clicks on continue button
    driver.find_element(By.ID, 'continue').click()

    time.sleep(random.uniform(3,10))

    # Selects password field on login page
    password_field_pizza_hobby = driver.find_element("name", "password") 
    # Enters password 
    password_field_pizza_hobby.send_keys(password)


    time.sleep(random.uniform(3,10))

    # Click sign in button to login the website.
    driver.find_element(By.ID, 'signInSubmit').click()

    # Wait for login process to complete and field "Zugehörigkeit" is loaded 
    time.sleep(random.uniform(3,10))

    # Click on field "Partnerprogramm"
    driver.find_element(By.XPATH, '//*[@id="navFooter"]/div[1]/div/div[3]/ul/li[4]/a').click()

    # Wait until page loaded 
    time.sleep(random.uniform(3,10))

    # Click on field "Berichte"
    driver.find_element(By.XPATH, '//*[@id="a-page"]/div[2]/nav/ul/li[5]/a').click()


    # Wait until page loaded 
    time.sleep(random.uniform(3,10))

    # Click on field "Einnahmen-Übersicht"
    driver.find_element(By.XPATH, '//*[@id="a-page"]/div[2]/nav/ul/li[5]/ul/li[2]/a').click()

    # Waits for 5 seconds until page is loaded 
    time.sleep(random.uniform(3,10))

    # Click on filter
    driver.find_element(By.XPATH, '//*[@id="ac-daterange-label-report-timeInterval"]').click()

    # Waits for 5 seconds until page is loaded 
    time.sleep(random.uniform(3,10))

    # Click on filter last 7 days
    driver.find_element(By.XPATH, '//*[@id="ac-daterange-radio-report-timeInterval-last_7_days"]/label/i').click()

    # Waits for 5 seconds until page is loaded 
    time.sleep(random.uniform(3,10))

    # Click on filter submit button
    driver.find_element(By.XPATH, '//*[@id="ac-daterange-ok-button-report-timeInterval-announce"]').click()

    # Waits for 5 seconds until page is loaded 
    time.sleep(random.uniform(3,10))
    
    # Clicks on field "Einnahmen"
    # driver.find_element(By.T, 'reports-commission-earnings-header').click()

    # Extracts commission table from website
    commissions_df = pd.read_html(driver.find_element(By.XPATH, '//*[@id="ac-report-commission-simple-earnings-tbl"]/div[5]/table').get_attribute('outerHTML'))[0]

    # Closes Webdriver
    driver.close()

    # Changes type of date column to datetime 
    commissions_df['DatumDatum'] = pd.to_datetime(commissions_df['DatumDatum'])

    # Changes timeformat such that only date is displayed
    commissions_df['DatumDatum'] = commissions_df['DatumDatum'].dt.date

    # Gets the commissions from yesterday 
    commissions = commissions_df['BestellwertÜber Produktlinks generierte Einnahmen'].loc[commissions_df['DatumDatum'] == pd.Timestamp('2023-06-20')]

    # Converts commissions to float
    commissions = (commissions.replace('[\€,]', '', regex=True).astype(float))/100

    # Sums up commissions
    total_commissions = commissions.sum()


    return total_commissions

In [ ]:
# login credentials fro pizzahobby.de
username = "##########" 
password = "**********"

# adds options for Chrome WebDriver
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')                   #<-- prevents to open test window
#chrome_options.add_argument('--no-sandbox')                #<--- don't know what this does
#chrome_options.add_argument('--disable-dev-shm-usage')      #<--- don't know what this does

# adds an random useragent to prevent recaptcha detection
ua = UserAgent()
userAgent = ua.random
chrome_options.add_argument(f'user-agent={userAgent}')

# Opens Chrome Browser
driver = webdriver.Chrome(chrome_options)


time.sleep(5)

# Minimizes Browser Windows
# driver.maximize_window()

# Navigates to URL
driver.get('https://www.amazon.de/ap/signin?openid.pape.max_auth_age=900&openid.return_to=https%3A%2F%2Fwww.amazon.de%2Fgp%2Fyourstore%2Fhome%3Fpath%3D%252Fgp%252Fyourstore%252Fhome%26signIn%3D1%26useRedirectOnSuccess%3D1%26action%3Dsign-out%26ref_%3Dnav_AccountFlyout_signout&openid.assoc_handle=deflex&openid.mode=checkid_setup&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0')

time.sleep(random.uniform(3,10))


# Selects username field on login page
username_field_pizza_hobby = driver.find_element("name", "email") 
# Enters username 
username_field_pizza_hobby.send_keys(username)

# Waits for 5 seconds to prevent suspicous behavior
time.sleep(random.uniform(3,10))

# Clicks on continue button
driver.find_element(By.ID, 'continue').click()

time.sleep(random.uniform(3,10))

# Selects password field on login page
password_field_pizza_hobby = driver.find_element("name", "password") 
# Enters password 
password_field_pizza_hobby.send_keys(password)


time.sleep(random.uniform(3,10))

# Click sign in button to login the website.
driver.find_element(By.ID, 'signInSubmit').click()

# Wait for login process to complete and field "Zugehörigkeit" is loaded 
time.sleep(random.uniform(3,10))

# Click on field "Partnerprogramm"
driver.find_element(By.XPATH, '//*[@id="navFooter"]/div[1]/div/div[3]/ul/li[4]/a').click()

# Wait until page loaded 
time.sleep(random.uniform(3,10))

# Click on field "Berichte"
driver.find_element(By.XPATH, '//*[@id="a-page"]/div[2]/nav/ul/li[5]/a').click()


# Wait until page loaded 
time.sleep(random.uniform(3,10))

# Click on field "Einnahmen-Übersicht"
driver.find_element(By.XPATH, '//*[@id="a-page"]/div[2]/nav/ul/li[5]/ul/li[2]/a').click()

# Waits for 5 seconds until page is loaded 
time.sleep(random.uniform(3,10))

# Click on filter
driver.find_element(By.XPATH, '//*[@id="ac-daterange-label-report-timeInterval"]').click()

# Waits for 5 seconds until page is loaded 
time.sleep(random.uniform(3,10))

# Click on filter last 7 days
driver.find_element(By.XPATH, '//*[@id="ac-daterange-radio-report-timeInterval-last_7_days"]/label/i').click()

# Waits for 5 seconds until page is loaded 
time.sleep(random.uniform(3,10))

# Click on filter submit button
driver.find_element(By.XPATH, '//*[@id="ac-daterange-ok-button-report-timeInterval-announce"]').click()

# Waits for 5 seconds until page is loaded 
time.sleep(random.uniform(3,10))

# Clicks on field "Einnahmen"
# driver.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[4]/div[2]/div[3]/div/div/div/div/div[1]/ul/span[2]/li/a').click()

# Extracts commission table from website
commissions_df = pd.read_html(driver.find_element(By.XPATH, '//*[@id="ac-report-commission-simple-earnings-tbl"]/div[5]/table').get_attribute('outerHTML'))[0]


# Closes Webdriver
# driver.close()

# Changes type of date column to datetime 
commissions_df['DatumDatum'] = pd.to_datetime(commissions_df['DatumDatum'])

# Changes timeformat such that only date is displayed
commissions_df['DatumDatum'] = commissions_df['DatumDatum'].dt.date

# Gets the commissions from yesterday 
commissions = commissions_df['BestellwertÜber Produktlinks generierte Einnahmen'].loc[commissions_df['DatumDatum'] == pd.Timestamp('2023-06-20')]

# Converts commissions to float
commissions = (commissions.replace('[\€,]', '', regex=True).astype(float))/100

# Sums up commissions
total_commissions = commissions.sum()



In [19]:
commissions_df = pd.read_html(driver.find_element(By.XPATH, '//*[@id="ac-report-commission-simple-earnings-tbl"]/div[5]/table').get_attribute('outerHTML'))[0]



In [24]:
commissions_df

,DatumDatum,KlicksAnzahl der Klicks auf Deine Partnerlinks.,"Anzahl der ausgelieferten Produkte(Direkt qualifizierte Käufe)(i)Die in Klammern angezeigten Werte geben Auskunft über die Anzahl aller qualifizierten Käufe, die über Deine direkten Partnerlinks getätigt wurden. Weitere Informationen findest Du in der Werbekostenerstattungstabelle für das Amazon EU-Partnerprogramm.","Summe der Umsätze(Direkt qualifizierte Käufe)(i)Die in Klammern angezeigten Werte geben Auskunft über die Anzahl aller qualifizierten Käufe, die über Deine direkten Partnerlinks getätigt wurden. Weitere Informationen findest Du in der Werbekostenerstattungstabelle für das Amazon EU-Partnerprogramm.","Summe der Vergütung(Direkt qualifizierte Käufe)(i)Die in Klammern angezeigten Werte geben Auskunft über die Anzahl aller qualifizierten Käufe, die über Deine direkten Partnerlinks getätigt wurden. Weitere Informationen findest Du in der Werbekostenerstattungstabelle für das Amazon EU-Partnerprogramm."
0,2023-06-23,16,2 (2),"€23,48 (€0,00)","€0,23 (€0,00)"
1,2023-06-22,15,2 (2),"€28,15 (€28,15)","€1,97 (€1,97)"
2,2023-06-20,24,3 (3),"€119,02 (€119,02)","€8,21 (€8,21)"
3,2023-06-19,31,2 (2),"€37,42 (€14,24)","€1,23 (€1,00)"
4,2023-06-18,32,3 (3),"€55,48 (€15,16)","€1,46 (€1,06)"


In [ ]:
# Changes type of date column to datetime 
commissions_df['DatumDatum'] = pd.to_datetime(commissions_df['DatumDatum'])

# Changes timeformat such that only date is displayed
commissions_df['DatumDatum'] = commissions_df['DatumDatum'].dt.date

# Gets the commissions from yesterday 
commissions = commissions_df['BestellwertÜber Produktlinks generierte Einnahmen'].loc[commissions_df['DatumDatum'] == pd.Timestamp('2023-06-20')]

# Converts commissions to float
commissions = (commissions.replace('[\€,]', '', regex=True).astype(float))/100

# Sums up commissions
total_commissions = commissions.sum()

# Agrieuro Commissions

In [5]:

# Gets commissions from pizza hobby
def agrieuro(date_yesterday):

    # login credentials fro pizzahobby.de
    username = "##########" 
    password = "**********"

    # adds options for Chrome WebDriver
    chrome_options = webdriver.ChromeOptions()
    #chrome_options.add_argument('--headless')                   #<-- prevents to open test window
    #chrome_options.add_argument('--no-sandbox')                #<--- don't know what this does
    #chrome_options.add_argument('--disable-dev-shm-usage')      #<--- don't know what this does

    # adds an random useragent to prevent recaptcha detection
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')

    # Opens Chrome Browser
    driver = webdriver.Chrome(chrome_options)


    time.sleep(5)

    # Minimizes Browser Windows
    # driver.maximize_window()

    # Navigates to URL
    driver.get('https://affiliation.agrieuro.com/affiliate-login?login')

    time.sleep(random.uniform(3,10))


    # Selects username field on login page
    username_field_pizza_hobby = driver.find_element("name", "afs_email") 
    # Enters username 
    username_field_pizza_hobby.send_keys(username)


    time.sleep(random.uniform(3,10))

    # Selects password field on login page
    password_field_pizza_hobby = driver.find_element("name", "afs_password") 
    # Enters password 
    password_field_pizza_hobby.send_keys(password)


    time.sleep(random.uniform(3,10))

    # Click sign in button to login the website.
    driver.find_element(By.XPATH, '//*[@id="afs_form_login"]/input[4]').click()

    # Wait for login process to complete and field "Zugehörigkeit" is loaded 
    time.sleep(random.uniform(3,10))

    # Click on field "Statistiken"
    driver.find_element(By.XPATH, '//*[@id="afs-menu"]/li[4]/a').click()

    # Wait until page loaded 
    time.sleep(random.uniform(3,10))

    # Click on field "Transaktionen"
    driver.find_element(By.XPATH, '//*[@id="afs-menu2"]/li[2]/a').click()

    # Wait until page loaded 
    time.sleep(random.uniform(3,10))

    # Extracts commission table from website
    commissions_df = pd.read_html(driver.find_element(By.XPATH, '//*[@id="afs_grid1"]').get_attribute('outerHTML'))[0]

    # Closes Webdriver
    driver.close()


    commissions_df.columns=commissions_df.iloc[0]
    commissions_df = commissions_df.drop(commissions_df.index[0])

    # Changes type of date column to datetime 
    commissions_df['Datum'] = pd.to_datetime(commissions_df['Datum'])

    # Changes timeformat such that only date is displayed
    commissions_df['Datum'] = commissions_df['Datum'].dt.date

    # Gets the commissions from yesterday 
    commissions = commissions_df['Kommission'].loc[commissions_df['Datum'] == pd.Timestamp('2023-06-20')]

    # Converts commissions to float
    commissions = (commissions.replace('[\€,]', '', regex=True).astype(float))

    # Sums up commissions
    total_commissions = commissions.sum()


    return total_commissions

# Send Mail

In [6]:
def send_mail(pizza_hobby_comission, amazon_comission, agrieuro_comission):


    # Serverdaten
    smtpServer = "smtp.web.de"
    smtpPort = 587

    # Zugangsdaten
    username = "##########"
    password = "**********"

    # Sender & Empfänger
    sender = "#############"
    reciever = "**********"

    total_commissions = pizza_hobby_comission + amazon_comission + agrieuro_comission

    # Converts floats to strings
    pizza_hobby_comission_str = str(pizza_hobby_comission)
    amazon_comission_str = str(amazon_comission)
    agrieuro_comission_str = str(agrieuro_comission)
    total_commissions_str = str(total_commissions)

    # Betreff & Inhalt
    subject = "Gestrige Kommissionen"
    body_html = '<html><body>'\
        '<h1 style="text-align:center"><span style="font-family:Verdana,Geneva,sans-serif">Kommissions&uuml;bersicht&nbsp;</span></h1>'\
        '<p>&nbsp;</p>'\
        '<p>&nbsp;</p>'\
        '<p>Pizza Hobby: '+ pizza_hobby_comission_str + '€''</p>'\
        '<p>Amazon: '+ amazon_comission_str + '€'' </p>'\
        '<p>Agrieuro: '+ agrieuro_comission_str + '€'' </p>'\
        '<p><strong><span style="font-size:16px">Gesamt: '+ total_commissions_str + '€''</span></strong></p>'\
        '</body></html>'

    # Message Objekt für die E-Mail, Mail wird zusammengefügt
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = reciever

    html_part = MIMEText(body_html, 'html')
    msg.attach(html_part)

    # Erzeugen einer Mail Session
    smtpObj = smtplib.SMTP(smtpServer, smtpPort)
    # Debuginformationen auf der Konsole ausgeben
    # smtpObj.set_debuglevel(1)


    # Wenn der Server eine Authentifizierung benötigt dann...
    smtpObj.starttls()
    smtpObj.login(username, password)


    # Sends Mail
    smtpObj.sendmail(sender, reciever, msg.as_string())



    return print("Mail send")

# Main

In [ ]:
# Gets yesterday's date
yesterdays_date = (datetime.today() - timedelta(1)).strftime('%Y-%m-%d')

# Gets commission from Pizza Hobby
commission_pizza_hobby = pizza_hobby(yesterdays_date)

# Gets commission from amazon
commission_amazon = amazon(yesterdays_date)

# Gets commission from Agrieuro
commission_agrieuro = agrieuro(yesterdays_date)

# Sends Mail with commission overview of yesterday
#send_mail(pizza_hobby_comission = commission_pizza_hobby, amazon_comission = commission_amazon, agrieuro_comission = commission_agrieuro)


print("Commission Amazon: ", commission_amazon)
print("Commission Pizza Hobby: ", commission_pizza_hobby)
print("Commission Agrieuro: ", commission_agrieuro)
print("Total Commission: ", commission_amazon + commission_agrieuro + commission_pizza_hobby)
